# LLaMA 3 SOTA Extractor EDA
This notebook provides two examples of SOTA extraction using LLaMA 3 in two steps. One example is a case where the paper does not report TDMS tuples and other example reports TDMS tuples. The two examples originate from the validation split of the SOTA dataset.

In [1]:
from latex_extractor import extract_content, extract_all_sections, get_section_name
from llama3_extractor import summarize_section, final_extraction
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

if torch.cuda.is_available():
    torch.set_default_device("cuda")
    device = "cuda"
else:
    torch.set_default_device("cpu")
    device = "cpu"

In [2]:
no_tdms_path = 'sota/dataset/validation/0705.1367/0705.1367.tex'
tdms_path = 'sota/dataset/validation/2110.01997v1/2110.01997v1.tex'
tdms_annotations_path = 'sota/dataset/validation/2110.01997v1/annotations.json'
with open(tdms_annotations_path) as f:
    tdms_annotations = f.read() 

content = extract_content(tdms_path)
content = content.replace('\t', ' ')
content = content.replace('\n', ' ')
sections = extract_all_sections(content)   

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
for section in sections:
    print(get_section_name(section))


Introduction
Related Works
The Proposed Method
Metrics
Experiments
Results
Conclusion
Summary
Training
Connectivity metric
Lane graph results


In [5]:
results = [section for section in sections if get_section_name(section) == "Results"][0]
results_summary = summarize_section(results, tokenizer, model)

print("Original Results Section:")
print(results)
print("-------------------------------------------")
print("Summary:")
print(results_summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Original Results Section:
{Results}    \caption{Precision/Recall vs thresholds. Thresholds are uniformly sampled in [0.01, 0.1] (normalized coordinates) with 0.01 increments. In our resolution, 0.01 corresponds to 50cm.  } Since our method produces a road network graph as well as dynamic object instance estimations, we divide the results into two subsections, studying those them individually.  \subsection{Lane graph}  The obtained results are provided in Tab.~\ref{tab:lane_compare} and Fig.~\ref{fig:prec-recall}, where our method achieves the best results in all metrics when compared to the baselines. The performance of PINET is lower, as expected, since the centerlines are obtained through processing lane boundaries. From the Poly(Est) vs Poly(GT) results, it can be seen that the localization of initial points is very difficult. Our method produces better precision-recall than Poly(Est), and the difference in detection and connectivity scores are significant. It is not surprising that

Summarize all sections

In [6]:
summaries = []
for section in sections:
    summaries.append(summarize_section(section, tokenizer, model))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

In [7]:
summary = " ".join(summaries)
tdms = final_extraction(summary, tokenizer, model)
print("Final TDMS:")
print(tdms)
print("Gold labels:")
print(tdms_annotations)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Final TDMS:
Based on the provided summaries, here are the leaderboards extracted:

[{'LEADERBOARD': {'Task': 'Lane-graph estimation', 'Dataset': 'Not explicitly mentioned', 'Metric': 'M-Pre', 'Score': '54.1'}},
 {'LEADERBOARD': {'Task': 'Lane-graph estimation', 'Dataset': 'Not explicitly mentioned', 'Metric': 'M-Rec', 'Score': '45.6'}},
 {'LEADERBOARD': {'Task': 'Lane-graph estimation', 'Dataset': 'Not explicitly mentioned', 'Metric': 'detection score', 'Score': '19.2'}},
 {'LEADERBOARD': {'Task': 'Lane-graph estimation', 'Dataset': 'Not explicitly mentioned', 'Metric': 'M-Prec', 'Score': '54.7'}},
 {'LEADERBOARD': {'Task': 'Lane-graph estimation', 'Dataset': 'Not explicitly mentioned', 'Metric': 'M-Rec', 'Score': '51.2'}},
 {'LEADERBOARD': {'Task': 'Lane-graph estimation', 'Dataset': 'Not explicitly mentioned', 'Metric': 'detection score', 'Score': '40.5'}},
 {'LEADERBOARD': {'Task': 'Lane-graph estimation', '
Gold labels:
[{'LEADERBOARD': {'Task': 'Lane Detection', 'Dataset': 'nuScen

Example with no TDMS tuples

In [8]:
content = extract_content(no_tdms_path)
content = content.replace('\t', ' ')
content = content.replace('\n', ' ')
sections_no_tdms = extract_all_sections(content)

In [9]:
summaries_no_tqdm = []
for section in sections_no_tdms:
    summaries_no_tqdm.append(summarize_section(section, tokenizer, model))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
summary_no_tdms = " ".join(summaries_no_tqdm)
no_tdms = final_extraction(summary_no_tdms, tokenizer, model)
print("Final result:")
print(no_tdms)
print("Gold label: unanswerable")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Final result:
[]
Gold label: unanswerable
